## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [5]:
!pip install pymorphy2

import pandas as pd
import numpy as np

import pymorphy2

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
train_data = pd.read_csv('/content/drive/MyDrive/news_train.txt', sep="\t", header=None, names=['topic', 'headline', 'info'])
test_data = pd.read_csv('/content/drive/MyDrive/news_test.txt', sep="\t", header=None, names=['topic', 'headline', 'info'])

In [10]:
train_data.head()

,topic,headline,info
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...


In [11]:
morph = pymorphy2.MorphAnalyzer()

In [12]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation + '«' + '»' + '—' else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation


russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])

def token(text):
    word_list = text.strip().split()
    res = []

    for word in word_list:
      p = morph.parse(word)[0]
      res.append(p.normal_form)
    
    tokens = [token for token in res if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
 for i in range(len(train_data['info'])):
    st = remove_punctuation(train_data['info'][i])
    st = remove_numbers(st)
    st = remove_multiple_spaces(st)

    st = token(st)

    train_data['info'][i] = st

In [14]:
 for i in range(len(test_data['info'])):
    st = remove_punctuation(test_data['info'][i])
    st = remove_numbers(st)
    st = remove_multiple_spaces(st)

    st = token(st)

    test_data['info'][i] = st

In [16]:
train_data.head()

,topic,headline,info
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,нападать вашингтон кэпиталзти александр овечки...
1,culture,Рекордно дорогую статую майя признали подделкой,власть мексика объявить подделка статуя майя п...
2,science,Samsung представила флагман в защищенном корпусе,южнокорейский samsung анонсировать защитить ве...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,контрольный дисциплинарный комитет кдк рфс сня...
4,media,Hopes & Fears объединится с The Village,интернет издание hopes fears объявить свой сли...


In [16]:
train_data.to_csv( "//content/drive/MyDrive/df_train.csv", index=False, encoding='utf-8-sig')

In [17]:
test_data.to_csv( "//content/drive/MyDrive/df_test.csv", index=False, encoding='utf-8-sig')

#Word2Vec

In [ ]:
!pip install gensim
from gensim.models import Word2Vec

In [19]:
info_list = [i.split() for i in train_data['info']]

In [20]:
model = Word2Vec(sentences=info_list, min_count=1, workers=4)

In [75]:
model.wv.most_similar('власть')

[('чиновник', 0.7827291488647461),
 ('сторона', 0.773669958114624),
 ('евросоюз', 0.7555724382400513),
 ('правительство', 0.7437057495117188),
 ('еврокомиссия', 0.7348254323005676),
 ('урегулировать', 0.7177960276603699),
 ('минюст', 0.7170125246047974),
 ('тегеран', 0.7161378264427185),
 ('киев', 0.7147305011749268),
 ('баньоля', 0.7086164355278015)]

In [22]:
label_encoder = LabelEncoder()
train_data['topic_target'] = label_encoder.fit_transform(train_data['topic'])

In [23]:
test_data['topic_target'] = label_encoder.transform(test_data['topic'])

#Models

In [40]:
ngram_range = (1, 2)
min_df = 10
max_df = 1.
max_features = 300

In [41]:
tfidf = TfidfVectorizer(encoding = 'utf-8',
                        ngram_range = ngram_range,
                        stop_words = None,
                        lowercase = False,
                        max_df = max_df,
                        min_df = min_df,
                        max_features = max_features,
                        norm = 'l2',
                        sublinear_tf = True)

In [42]:
feature_train = tfidf.fit_transform(train_data['info']).toarray()
labels_train = train_data['topic_target']

feature_test = tfidf.transform(test_data['info']).toarray()
labels_test = test_data['topic_target']

#NB

In [67]:
from sklearn.naive_bayes import MultinomialNB

param_grid = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]}
model_nb = MultinomialNB()
grid = GridSearchCV(model_nb, param_grid, cv=5, verbose=5)
best = grid.fit(feature_train, labels_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ........................alpha=0.01;, score=0.738 total time=   0.0s
[CV 2/5] END ........................alpha=0.01;, score=0.739 total time=   0.0s
[CV 3/5] END ........................alpha=0.01;, score=0.741 total time=   0.0s
[CV 4/5] END ........................alpha=0.01;, score=0.729 total time=   0.0s
[CV 5/5] END ........................alpha=0.01;, score=0.735 total time=   0.0s
[CV 1/5] END .........................alpha=0.1;, score=0.738 total time=   0.0s
[CV 2/5] END .........................alpha=0.1;, score=0.739 total time=   0.0s
[CV 3/5] END .........................alpha=0.1;, score=0.741 total time=   0.0s
[CV 4/5] END .........................alpha=0.1;, score=0.729 total time=   0.0s
[CV 5/5] END .........................alpha=0.1;, score=0.737 total time=   0.0s
[CV 1/5] END .........................alpha=0.5;, score=0.737 total time=   0.0s
[CV 2/5] END .........................alpha=0.5;,

In [76]:
model_predictions_nb = best.predict(feature_test)
print(accuracy_score(labels_test, model_predictions_nb))

0.751


#SVM

In [ ]:
from sklearn.svm import SVC

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
model_sv = SVC()
grid_1 = GridSearchCV(model_sv, param_grid, cv=3, verbose=2)
bestF = grid_1.fit(feature_train, labels_train)

In [64]:
model_predictions_svm = bestF.predict(feature_test)
print(accuracy_score(labels_test, model_predictions_svm))

0.8156666666666667
